In [112]:
from TableExtract import HTMLTableParser
import pandas as pd
from bs4 import BeautifulSoup
import requests

import warnings
warnings.filterwarnings('ignore')

import time

In [90]:
#url = 'http://www.cfbstats.com/2017/team/8/passing/defense/gamelog.html'
#url = 'http://www.cfbstats.com/2017/team/31/interception/gamelog.html'
#url = 'http://www.cfbstats.com/2017/team/234/passing/offense/gamelog.html'

In [68]:
hp = HTMLTableParser()
df = hp.parse_url(url)[0][1]
df

Date               Opponent Surface   Result   Att  \
0                     09/02/17            + 1 Alabama    Turf   L 7-24  33.0   
1                     09/23/17  23 North Carolina St.   Grass  L 21-27  39.0   
2                     09/30/17          @ Wake Forest    Turf  W 26-19  21.0   
3                     10/07/17        13 Miami (Fla.)   Grass  L 20-24  28.0   
4                     10/14/17                 @ Duke   Grass  W 17-10  21.0   
5                     10/21/17             Louisville   Grass  L 28-31  28.0   
6                     10/27/17       @ Boston College    Turf   L 3-35  27.0   
7                     11/04/17               Syracuse   Grass  W 27-24  19.0   
8                     11/11/17            @ 4 Clemson   Grass  L 14-31  32.0   
9                     11/18/17           Delaware St.   Grass   W 77-6  24.0   
10                    11/25/17              @ Florida   Grass  W 38-22  21.0   
11                    12/02/17             La.-Monroe   Grass  W 42-10  23.0   
12                    12/27/17       + Southern Miss.    Turf  W 42-13  29.0   
13                                             Totals     345      202  58.6   
14  @ : Away, + : Neutral Site                    NaN     NaN      NaN   NaN   

      Comp  Pct.  Yards  Yards/Att      TD  Int  Rating  
0     19.0  57.6  210.0        6.4    1.00  2.0  108.92  
1     22.0  56.4  278.0        7.1    1.00  0.0  124.75  
2     11.0  52.4  121.0        5.8    1.00  0.0  116.49  
3     17.0  60.7  203.0        7.3    2.00  2.0  130.89  
4     18.0  85.7  197.0        9.4    1.00  2.0  161.17  
5     16.0  57.1  248.0        8.9    2.00  2.0  140.82  
6     12.0  44.4  149.0        5.5    0.00  1.0   83.39  
7     12.0  63.2  136.0        7.2    1.00  0.0  140.66  
8     13.0  40.6  208.0        6.5    1.00  1.0   99.29  
9     18.0  75.0  236.0        9.8    4.00  0.0  212.60  
10    10.0  47.6  128.0        6.1    2.00  1.0  120.73  
11    15.0  65.2  199.0        8.7    1.00  2.0  134.86  
12    19.0  65.5  238.0        8.2    4.00  0.0  179.98  
13  2551.0   7.4   21.0       13.0  133.21  NaN     NaN  
14     NaN   NaN    NaN        NaN     NaN  NaN     NaN

In [11]:
#FUNC: Drop last two rows
def dropTail(table):
    new_table = table.drop(table.tail(2).index)
    return new_table

In [14]:
#FUNC: Add game locations
def homeAway(table):
    game_locations = []

    for row in table['Opponent']:
        if '@' in row:
            game_locations.append('away')
        elif '+' in row:
            game_locations.append('neutral')
        else:
            game_locations.append('home')

    table['Location'] = game_locations
    return table

In [93]:
#FUNC: Update opponent names
def opponentName(table, tableURL):
    response = requests.get(tableURL)
    soup = BeautifulSoup(response.text, 'lxml')
    rows = soup.select('td[class="opponent"]')
    
    opponent_names = []
    
    for row in rows:
        try:
            opponent_names.append(row.find('a').text)
        except:
            opponent_names.append('FCS')
    
    table['Opponent'] = opponent_names
    return table

In [146]:
#FUNC: Add team name column
def teamName(table, name, team_id):
    table.insert(loc=1, column='Team', value=name)
    table.insert(loc=1, column='Team_ID', value=team_id)
    return table

In [69]:
df = dropTail(df)
df = homeAway(df)
df = opponentName(url, df)
df.head()

Date            Opponent Surface   Result   Att  Comp  Pct.  Yards  \
0  09/02/17             Alabama    Turf   L 7-24  33.0  19.0  57.6  210.0   
1  09/23/17  North Carolina St.   Grass  L 21-27  39.0  22.0  56.4  278.0   
2  09/30/17         Wake Forest    Turf  W 26-19  21.0  11.0  52.4  121.0   
3  10/07/17        Miami (Fla.)   Grass  L 20-24  28.0  17.0  60.7  203.0   
4  10/14/17                Duke   Grass  W 17-10  21.0  18.0  85.7  197.0   

   Yards/Att   TD  Int  Rating Location  
0        6.4  1.0  2.0  108.92  neutral  
1        7.1  1.0  0.0  124.75     home  
2        5.8  1.0  0.0  116.49     away  
3        7.3  2.0  2.0  130.89     home  
4        9.4  1.0  2.0  161.17     away

In [148]:
#FUNC: Extract offensive passing
def extractOffensivePassing(table):
    #table[['Result','Score']] = df['Result'].str.split(' ',expand=True)
    
    table = table[['Date', 'Team_ID', 'Team', 'Opponent', 'Result', 'Att', 'Yards', 'Int', 'Location']]
    table[['Att', 'Yards', 'Int']] = table[['Att', 'Yards', 'Int']].astype('int64')
    table = table.rename(columns={'Att':'Off Pass Att', 'Yards':'Off Pass Yards', 'Int':'Off Int'})
    
    return table

In [70]:
df = extractOffensivePassing(df)
df.head()

Date            Opponent Result  Score  Off Pass Att  Off Pass Yards  \
0  09/02/17             Alabama      L   7-24            33             210   
1  09/23/17  North Carolina St.      L  21-27            39             278   
2  09/30/17         Wake Forest      W  26-19            21             121   
3  10/07/17        Miami (Fla.)      L  20-24            28             203   
4  10/14/17                Duke      W  17-10            21             197   

   Off Int Location  
0        2  neutral  
1        0     home  
2        0     away  
3        2     home  
4        2     away

In [71]:
#FUNC: Extract defensive passing
def extractDefensivePassing(table):
    table = table[['Att', 'Yards']]
    table[['Att', 'Yards']] = table[['Att', 'Yards']].astype('int64')
    table = table.rename(columns={'Att':'Def Pass Att', 'Yards':'Def Pass Yards'})
    
    return table

In [72]:
#FUNC: Extract offensive rushing
def extractOffensiveRushing(table):
    table = table[['Att', 'Yards']]
    table[['Att', 'Yards']] = table[['Att', 'Yards']].astype('int64')
    table = table.rename(columns={'Att':'Off Rush Att', 'Yards':'Off Rush Yards'})
    
    return table

In [117]:
#FUNC: Extract defensive rushing
def extractDefensiveRushing(table):
    table = table[['Att', 'Yards']]
    table[['Att', 'Yards']] = table[['Att', 'Yards']].astype('int64')
    table = table.rename(columns={'Att':'Def Rush Att', 'Yards':'Def Rush Yards'})
    
    return table

In [118]:
#FUNC: Extract defensive sacks
def extractDefensiveSacks(table):
    table = table[['Sacks', 'Sack Yards']]
    table[['Sacks', 'Sack Yards']] = table[['Sacks', 'Sack Yards']].astype('int64')
    table = table.rename(columns={'Sacks':'Def Sacks', 'Sack Yards':'Def Sack Yards'})
    
    return table

In [139]:
#FUNC: Extract total offensive plays
def extractTotalPlays(table):
    table = table[['Plays']]
    table['Plays'] = table['Plays'].astype('int64')
    table = table.rename(columns={'Plays':'Off Plays'})
    
    return table

In [120]:
#FUNC: Extract penalties
def extractPenalties(table):
    table = table[['Pen.', 'Yards']]
    table[['Pen.', 'Yards']] = table[['Pen.', 'Yards']].astype('int64')
    table = table.rename(columns={'Pen.':'Pen', 'Yards':'Pen Yards'})
    
    return table

In [98]:
#------------------------------------------#
#--------------Concat example--------------#
#------------------------------------------#
test1 = {'A':[1,2], 'B':[3,4]}
test2 = {'C':[5,6], 'D':[7,8]}

tf1 = pd.DataFrame(test1)
tf2 = pd.DataFrame(test2)

frames = [tf1, tf2]
result = pd.concat(frames, axis=1)

In [104]:
#FUNC: Call HTMLTableParser
def tableExtractor(url):
    
    hp = HTMLTableParser()
    table = hp.parse_url(url)[0][1]
    
    new_table = table.drop(table.tail(2).index)
    
    return new_table

In [150]:
years = ['2017', '2016', '2015', '2014', '2013', '2012', '2011', '2010', '2009', '2008']
teams = pd.read_csv('team_ids.csv')
#teams = team_ids.head(2)
#teams

In [165]:
start_time = time.time()

for year in years:
    
    all_frames = []

    for i in range(len(teams)):
        url_off_pass = f'http://www.cfbstats.com/{year}/team/{teams["team_id"][i]}/passing/offense/gamelog.html'
        url_def_pass = f'http://www.cfbstats.com/{year}/team/{teams["team_id"][i]}/passing/defense/gamelog.html'
        url_off_rush = f'http://www.cfbstats.com/{year}/team/{teams["team_id"][i]}/rushing/offense/gamelog.html'
        url_def_rush = f'http://www.cfbstats.com/{year}/team/{teams["team_id"][i]}/rushing/defense/gamelog.html'
        url_def_sack = f'http://www.cfbstats.com/{year}/team/{teams["team_id"][i]}/sack/gamelog.html'
        url_tot_play = f'http://www.cfbstats.com/{year}/team/{teams["team_id"][i]}/total/offense/gamelog.html'
        url_pen = f'http://www.cfbstats.com/{year}/team/{teams["team_id"][i]}/penalty/offense/gamelog.html'
        
        r = requests.get(url_off_pass)
        
        if r.status_code == 404:
            print(f'404 NOT FOUND: {url_off_pass}')
            continue

        df_off_pass = tableExtractor(url_off_pass)
        df_off_pass = homeAway(df_off_pass)
        df_off_pass = opponentName(df_off_pass, url_off_pass)
        df_off_pass = teamName(df_off_pass, teams['team_name'][i], teams['team_id'][i])
        df_off_pass = extractOffensivePassing(df_off_pass)
        #print('df_off_pass CREATED: {teams["team_name"][i]}')

        df_def_pass = tableExtractor(url_def_pass)
        df_def_pass = extractDefensivePassing(df_def_pass)
        #print(f'df_def_pass CREATED: {teams["team_name"][i]}')

        df_off_rush = tableExtractor(url_off_rush)
        df_off_rush = extractOffensiveRushing(df_off_rush)
        #print(f'df_off_rush CREATED: {teams["team_name"][i]}')

        df_def_rush = tableExtractor(url_def_rush)
        df_def_rush = extractDefensiveRushing(df_def_rush)
        #print(f'df_def_rush CREATED: {teams["team_name"][i]}')

        df_def_sack = tableExtractor(url_def_sack)
        df_def_sack = extractDefensiveSacks(df_def_sack)
        #print(f'df_def_sack CREATED: {teams["team_name"][i]}')

        df_tot_play = tableExtractor(url_tot_play)
        df_tot_play = extractTotalPlays(df_tot_play)
        #print(f'df_tot_play CREATED: {teams["team_name"][i]}')

        df_pen = tableExtractor(url_pen)
        df_pen = extractPenalties(df_pen)
        #print(f'df_pen CREATED: {teams["team_name"][i]}')

        frames = [df_off_pass, df_off_rush, df_def_pass, df_def_rush, df_def_sack, df_tot_play, df_pen]
        result = pd.concat(frames, axis=1)
        #print(f'frames concatenated!!')
        
        
        all_frames.append(result)
        print(f'Team {i} for {year}')

    final = pd.concat(all_frames)
    final.to_csv(f'test_scrape_{year}.csv', index=False)
    print('-------------------------')

total_time = time.time()-start_time
print(f'Time elapsed: {total_time}')

Team 0 for 2017
Team 1 for 2017
Team 2 for 2017
Team 3 for 2017
Team 4 for 2017
Team 5 for 2017
Team 6 for 2017
Team 7 for 2017
Team 8 for 2017
Team 9 for 2017
Team 10 for 2017
Team 11 for 2017
Team 12 for 2017
Team 13 for 2017
Team 14 for 2017
Team 15 for 2017
Team 16 for 2017
Team 17 for 2017
Team 18 for 2017
Team 19 for 2017
Team 20 for 2017
Team 21 for 2017
Team 22 for 2017
Team 23 for 2017
Team 24 for 2017
Team 25 for 2017
Team 26 for 2017
Team 27 for 2017
Team 28 for 2017
Team 29 for 2017
Team 30 for 2017
Team 31 for 2017
Team 32 for 2017
Team 33 for 2017
Team 34 for 2017
Team 35 for 2017
Team 36 for 2017
Team 37 for 2017
Team 38 for 2017
Team 39 for 2017
Team 40 for 2017
Team 41 for 2017
Team 42 for 2017
Team 43 for 2017
Team 44 for 2017
Team 45 for 2017
Team 46 for 2017
Team 47 for 2017
Team 48 for 2017
Team 49 for 2017
Team 50 for 2017
Team 51 for 2017
Team 52 for 2017
Team 53 for 2017
Team 54 for 2017
Team 55 for 2017
Team 56 for 2017
Team 57 for 2017
Team 58 for 2017
Team 59

Team 62 for 2014
Team 63 for 2014
Team 64 for 2014
Team 65 for 2014
Team 66 for 2014
Team 67 for 2014
Team 68 for 2014
Team 69 for 2014
Team 70 for 2014
Team 71 for 2014
Team 72 for 2014
Team 73 for 2014
Team 74 for 2014
Team 75 for 2014
Team 76 for 2014
Team 77 for 2014
Team 78 for 2014
Team 79 for 2014
Team 80 for 2014
Team 81 for 2014
Team 82 for 2014
Team 83 for 2014
Team 84 for 2014
Team 85 for 2014
Team 86 for 2014
Team 87 for 2014
Team 88 for 2014
Team 89 for 2014
Team 90 for 2014
Team 91 for 2014
Team 92 for 2014
Team 93 for 2014
Team 94 for 2014
Team 95 for 2014
Team 96 for 2014
Team 97 for 2014
Team 98 for 2014
Team 99 for 2014
Team 100 for 2014
Team 101 for 2014
Team 102 for 2014
Team 103 for 2014
Team 104 for 2014
Team 105 for 2014
Team 106 for 2014
Team 107 for 2014
Team 108 for 2014
Team 109 for 2014
Team 110 for 2014
Team 111 for 2014
Team 112 for 2014
Team 113 for 2014
Team 114 for 2014
Team 115 for 2014
Team 116 for 2014
Team 117 for 2014
Team 118 for 2014
Team 119 for

Team 81 for 2011
404 NOT FOUND: http://www.cfbstats.com/2011/team/523/passing/offense/gamelog.html
Team 83 for 2011
Team 84 for 2011
Team 85 for 2011
Team 86 for 2011
Team 87 for 2011
Team 88 for 2011
Team 89 for 2011
Team 90 for 2011
Team 91 for 2011
Team 92 for 2011
404 NOT FOUND: http://www.cfbstats.com/2011/team/646/passing/offense/gamelog.html
Team 94 for 2011
Team 95 for 2011
Team 96 for 2011
Team 97 for 2011
Team 98 for 2011
Team 99 for 2011
Team 100 for 2011
Team 101 for 2011
Team 102 for 2011
Team 103 for 2011
404 NOT FOUND: http://www.cfbstats.com/2011/team/670/passing/offense/gamelog.html
Team 105 for 2011
Team 106 for 2011
Team 107 for 2011
Team 108 for 2011
Team 109 for 2011
Team 110 for 2011
Team 111 for 2011
Team 112 for 2011
Team 113 for 2011
Team 114 for 2011
Team 115 for 2011
404 NOT FOUND: http://www.cfbstats.com/2011/team/706/passing/offense/gamelog.html
Team 117 for 2011
Team 118 for 2011
Team 119 for 2011
Team 120 for 2011
Team 121 for 2011
Team 122 for 2011
Team 

Team 55 for 2008
404 NOT FOUND: http://www.cfbstats.com/2008/team/400/passing/offense/gamelog.html
Team 57 for 2008
Team 58 for 2008
Team 59 for 2008
Team 60 for 2008
Team 61 for 2008
Team 62 for 2008
Team 63 for 2008
Team 64 for 2008
Team 65 for 2008
Team 66 for 2008
Team 67 for 2008
Team 68 for 2008
Team 69 for 2008
Team 70 for 2008
Team 71 for 2008
Team 72 for 2008
Team 73 for 2008
Team 74 for 2008
Team 75 for 2008
Team 76 for 2008
Team 77 for 2008
Team 78 for 2008
Team 79 for 2008
Team 80 for 2008
Team 81 for 2008
404 NOT FOUND: http://www.cfbstats.com/2008/team/523/passing/offense/gamelog.html
Team 83 for 2008
Team 84 for 2008
Team 85 for 2008
Team 86 for 2008
Team 87 for 2008
Team 88 for 2008
Team 89 for 2008
Team 90 for 2008
Team 91 for 2008
Team 92 for 2008
404 NOT FOUND: http://www.cfbstats.com/2008/team/646/passing/offense/gamelog.html
Team 94 for 2008
Team 95 for 2008
Team 96 for 2008
Team 97 for 2008
Team 98 for 2008
Team 99 for 2008
Team 100 for 2008
Team 101 for 2008
Team